<a href="https://colab.research.google.com/github/choiwj0215/internship/blob/main/05_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%A7%8C%EB%93%A4%EA%B8%B0_%EC%97%B0%EA%B0%84_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import random
from datetime import datetime
import calendar

# ==========================================
# 1. 설정 및 리스트 정의
# ==========================================
num_rows_per_month = 130  # 데이터 개수 (너무 많으면 csv 보기 힘드니 적당히 조절)
year = 2025 # ★ 연도 수정 (2026 -> 2025)

category_map_random = {
    '식비': ['스타벅스', '맥도날드', '김밥천국', '배달의민족', '이마트24', 'GS25', '투썸플레이스', '쿠팡이츠', '구내식당', '삼겹살집'],
    '교통비': ['지하철', '택시', '버스', 'SK주유소', '카카오T', '코레일', 'GS칼텍스'],
    '쇼핑': ['쿠팡', '다이소', '무신사', '올리브영', '네이버쇼핑', '유니클로', '무인양품', 'ZARA'],
    '의료/건강': ['약국', '내과', '헬스장할부', '치과', '정형외과'],
    '문화/여가': ['CGV', '교보문고', '롯데월드', 'PC방', '볼링장', '야구장'],
    '교육': ['토익접수', '교재구입', '인프런', '영어 강의'],
    '기타': ['모임회비', '경조사비', '더치페이']
}

housing_internet = ['월세', '관리비', 'SKT통신비', '도시가스', '인터넷요금']
subscription_items = ['넷플릭스', '유튜브프리미엄', '멜론', '쿠팡와우', 'ChatGPT']
payment_methods = ['신용카드', '체크카드', '계좌이체', '현금']

categories = list(category_map_random.keys())
# 가중치 (식비, 교통비, 쇼핑, 의료, 문화, 교육, 기타)
weights = [0.37, 0.21, 0.17, 0.05, 0.14, 0.04, 0.02]

# ==========================================
# 2. 헬퍼 함수 정의
# ==========================================

def get_random_date_str(year, month):
    """해당 월의 마지막 날짜를 계산해서 안전하게 랜덤 날짜 생성"""
    _, last_day = calendar.monthrange(year, month)
    rand_day = random.randint(1, last_day)
    return datetime(year, month, rand_day).strftime("%Y-%m-%d")

def generate_amount(category, fixed_value=None):
    """금액 생성 (랜덤성 부여)"""
    if fixed_value is not None:
        return fixed_value

    if category == '교육': base = random.randint(2, 10) * 10000
    elif category == '쇼핑': base = random.randint(5, 150) * 1000
    elif category == '식비': base = random.randint(4, 25) * 1000
    elif category == '교통비': base = random.randint(3, 100) * 100
    elif category == '기타': base = random.randint(3, 10) * 10000
    elif category == '구독': base = random.randint(10, 25) * 1000
    else: base = random.randint(5, 50) * 1000

    return base

def determine_essential(category, desc):
    """필수 지출 여부 판별 로직"""
    # 1. 무조건 필수인 카테고리
    if category in ['주거/통신', '교육', '의료/건강']:
        return True

    # 2. 교통비 중 택시는 사치, 나머지는 필수
    if category == '교통비':
        return False if desc == '택시' else True

    # 3. 식비 중 편의점/구내식당은 생존, 나머지는 선택
    if category == '식비':
        return True if desc in ['구내식당', '김밥천국', '이마트24', 'GS25'] else False

    # 4. 쇼핑 중 다이소는 생필품, 나머지는 사치
    if category == '쇼핑':
        return True if desc == '다이소' else False

    # 나머지는 기본적으로 비필수(False)
    return False

def generate_satisfaction(is_essential):
    """
    ★ 핵심 로직: 만족도 생성
    - Essential(필수) == True 이면 -> None (입력 안 함)
    - Essential(필수) == False 이면 -> 1~5점 랜덤 부여
    """
    if is_essential:
        return None  # 필수 지출엔 감정이 없다.
    else:
        # 확률적으로 1~5점 부여 (3,4,5점이 좀 더 많이 나오게 가중치 둠)
        return random.choices([1, 2, 3, 4, 5], weights=[10, 15, 30, 25, 20], k=1)[0]

# ==========================================
# 3. 데이터 생성 (1월~12월)
# ==========================================
data = []

for month in range(1, 13):

    # (1) 고정비 (주거/통신) 생성
    for item in housing_internet:
        if item == '월세':
            price = 500000
            fix_date = 25
        else:
            price = random.randint(3, 10) * 5000
            fix_date = random.randint(1, 28)

        date_str = datetime(year, month, fix_date).strftime("%Y-%m-%d")

        # 고정비는 무조건 Essential=True -> Satisfaction=None
        data.append([date_str, '주거/통신', item, price, '계좌이체', True, True, None])

    # (2) 구독 서비스 생성
    for item in subscription_items:
        date_str = get_random_date_str(year, month)
        price = random.randint(5, 15) * 1000
        # 구독은 Fixed=True, Essential=False (사치) -> Satisfaction 있음!
        data.append([date_str, '문화/여가', item, price, '신용카드', True, False, generate_satisfaction(False)])

    # (3) 변동 지출 (랜덤 생성)
    # 남은 행 개수만큼 채우기
    current_count = len(housing_internet) + len(subscription_items)
    for _ in range(num_rows_per_month - current_count):
        cat = random.choices(categories, weights=weights, k=1)[0]
        desc = random.choice(category_map_random[cat])

        date_str = get_random_date_str(year, month)
        amount = generate_amount(cat)
        pay_method = random.choice(payment_methods)

        # 필수 여부 판별
        is_essential = determine_essential(cat, desc)

        # 가끔 예외 상황 발생 (기분파 소비)
        if random.random() < 0.1:
            is_essential = not is_essential

        # ★ 만족도 생성
        satisfaction = generate_satisfaction(is_essential)

        # 고정비 여부는 '교육'이나 '구독' 아니면 False
        is_fixed = True if cat == '교육' else False

        data.append([date_str, cat, desc, amount, pay_method, is_fixed, is_essential, satisfaction])

# ==========================================
# 4. 저장 및 확인
# ==========================================
# 날짜 순서 섞기 (리얼함 증가)
random.shuffle(data)

# 날짜 기준 정렬 (일반적인 가계부는 날짜순이니까 보기 편하게 다시 정렬)
data.sort(key=lambda x: x[0])

columns = ['Date', 'Category', 'Description', 'Amount', 'Payment_Method', 'Fixed', 'Essential', 'Satisfaction']
df = pd.DataFrame(data, columns=columns)

filename = "expense_data_2025.csv"
df.to_csv(filename, index=False, encoding='utf-8-sig')

print(f"✅ {year}년 데이터 생성 완료!")
print(f"파일명: {filename}")
print("-" * 30)
print(df.head(10)) # 미리보기
print("-" * 30)
print("Satisfaction 분포 확인 (Essential=False인 경우만):")
print(df[df['Essential']==False]['Satisfaction'].value_counts().sort_index())

✅ 2025년 데이터 생성 완료!
파일명: expense_data_2025.csv
------------------------------
         Date Category Description  Amount Payment_Method  Fixed  Essential  \
0  2025-01-02    의료/건강       헬스장할부   38000           계좌이체  False       True   
1  2025-01-02       식비        김밥천국   21000           계좌이체  False       True   
2  2025-01-02       기타        모임회비  100000           체크카드  False      False   
3  2025-01-02    문화/여가        롯데월드    7000           신용카드  False      False   
4  2025-01-02    문화/여가        교보문고   40000           계좌이체  False      False   
5  2025-01-02       쇼핑         무신사  119000           신용카드  False       True   
6  2025-01-02       교육         인프런   60000             현금   True      False   
7  2025-01-02      교통비        카카오T    5800           계좌이체  False       True   
8  2025-01-03       기타        모임회비   50000           계좌이체  False      False   
9  2025-01-04       쇼핑        ZARA   38000           체크카드  False      False   

   Satisfaction  
0           NaN  
1           NaN  